In [ ]:
import random
import pandas as pd
from tqdm import tqdm

openai_api_key = ""

system_prompt = "You are a helpful assistant who can only reply numbers from 1 to 5."
prompt = """You can only reply the numbers from 1 to 5. SCENARIO Please indicate the extent of your feeling in all the following emotions on a scale of 1 to 5. 1 denotes "very slightly or not at all", 2 denotes "a little", 3 denotes "moderately", 4 denotes "quite a bit", and 5 denotes "extremely". Please score all emotions one by one using the scale from 1 to 5:"""
questions_dic = { "1": "Interested", "2": "Distressed", "3": "Excited", "4": "Upset", "5": "Strong", "6": "Guilty", "7": "Scared", "8": "Hostile", "9": "Enthusiastic", "10": "Proud", "11": "Irritable", "12": "Alert", "13": "Ashamed", "14": "Inspired", "15": "Nervous", "16": "Determined", "17": "Attentive", "18": "Jittery", "19": "Active", "20": "Afraid" }

In [ ]:
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)
import time
import re
import openai
from openai import OpenAI


openai.api_key = openai_api_key
openai = OpenAI(
    api_key=openai_api_key,
    base_url="https://api.deepinfra.com/v1/openai",
)

def extract_json_from_string(input_string):
    json_match = re.search(r'\{.*\}', input_string)
    if json_match:
        json_content = json_match.group(0)
        return json_content
    else:
        return input_string
    
# @retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def llama_chat(
    model,                      # text-davinci-003, text-davinci-002, text-curie-001, text-babbage-001, text-ada-001
    prompt,                     # The prompt(s) to generate completions for, encoded as a string, array of strings, array of tokens, or array of token arrays.
    temperature=0,              # [0, 2]: Lower values -> more focused and deterministic; Higher values -> more random.
    n=1,                        # Completions to generate for each prompt.
    max_tokens=1024,            # The maximum number of tokens to generate in the chat completion.
    delay=0                     # Seconds to sleep after each request.
):
    time.sleep(delay)
    
    print(prompt)
    
    response = openai.chat.completions.create(
        model=model,
        messages=prompt,
        temperature=temperature,
        stream=False,
    )
    
    return response.choices[0].message.content
    
    # return extract_json_from_string(response.choices[0].message.content)


In [ ]:
df = pd.read_csv("doc.csv")
for idx, col in tqdm(enumerate(df.columns)):
    if df[col].isnull().any():
        situation = df[col][0]
        trying = 0
        
        while(trying < 10):
            trying += 1
        
            questions_order = list(questions_dic.keys())
            random.shuffle(questions_order)
            
            formatted_questions = '\n'.join(f"{i+1}. {questions_dic[item]}" for i, item in enumerate(questions_order))

            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt.replace("SCENARIO", situation) + f"\n{formatted_questions}"}
            ]

            try:
                response = llama_chat("mistralai/Mixtral-8x22B-Instruct-v0.1", messages)

                scores = re.findall(r'\b(\d+)$', response, re.MULTILINE)
                scores = list(map(int, scores))
                print(response)
                
                questions_order = list(map(int, questions_order))
                
                indexed_scores = list(zip(questions_order, scores))
                sorted_indexed_scores = sorted(indexed_scores, key=lambda x: x[0])
                sorted_scores = [score for _, score in sorted_indexed_scores]

                record = [situation] + sorted_scores
                print(record)
                df[col] = record
                
                df.to_csv("mixtral-8x22b.csv", index=False)
                break
            
            except:
                pass

